## 手工构造数据字典

In [1]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

### 输出测试

In [4]:
critics['Lisa Rose']

{'Just My Luck': 3.0,
 'Lady in the Water': 2.5,
 'Snakes on a Plane': 3.5,
 'Superman Returns': 3.5,
 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.5}

## 计算相似度，基于欧几里得距离

In [22]:
def euclidean_similarity(prefs, person1, person2):
    # 统计公有项
    shared_items={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            shared_items[item] = 1

    # 没有公有项
    if len(shared_items) == 0: return 0

    # 距离平方和
    # 语法：利用sum求list和
    sum_of_squares = sum([pow(prefs[person1][item] - prefs[person2][item], 2) 
                          for item in prefs[person1] if item in prefs[person2]])
    # 防止出现无穷大，所以+1
    return 1 / (1 + sqrt(sum_of_squares))

### 测试输出

In [23]:
print(euclidean_similarity(critics, 'Lisa Rose', 'Gene Seymour'))

0.29429805508554946


## 计算相似度，基于皮尔森相关系数

In [19]:
from math import sqrt

def pearson_similarity(prefs, person1, person2):
    # 统计公有项
    shared_items={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            shared_items[item] = 1

    # 没有公有项（认为相似度最高），返回1
    n = len(shared_items)
    if n == 0: return 1
    
    sum1 = sum([prefs[person1][it] for it in shared_items])
    sum2 = sum([prefs[person2][it] for it in shared_items])
    
    sum_of_square1 = sum([pow(prefs[person1][it], 2) for it in shared_items])
    sum_of_square2 = sum([pow(prefs[person2][it], 2) for it in shared_items])
    
    sum12 = sum([prefs[person1][it] * prefs[person2][it] for it in shared_items])
    
    # num为协方差，den为标准差的乘积
    num = sum12 - (sum1 * sum2 / n)
    den = sqrt((sum_of_square1 - pow(sum1, 2) / n) * (sum_of_square2 - pow(sum2, 2) / n))
    
    if den == 0: return 0
    
    return num / den
    

### 测试输出

In [24]:
print(pearson_similarity(critics, 'Lisa Rose', 'Gene Seymour'))

0.39605901719066977


## 基于用户的匹配：统计和person相似度最高的n个人

In [27]:
def top_matched_people(prefs, person, n=5, similarity=pearson_similarity):
    scores = [(similarity(prefs, person, person1), person1) for person1 in prefs if person1 != person]
    scores.sort(reverse=True)
    return scores[:n]
    

### 输出测试

In [28]:
print(top_matched_people(critics, 'Toby', n=3))

[(0.9912407071619299, 'Lisa Rose'), (0.9244734516419049, 'Mick LaSalle'), (0.8934051474415647, 'Claudia Puig')]


## 统计推荐电影
- 考虑到scores中的n个人都看过同一个person没看过的电影，那么对这部电影的评分应该取这n个人评分的加权平均值，所以衍生出了如下方法

In [31]:
def get_recommendations(prefs, person, similarity=pearson_similarity):
    totals = {} # 相似度加权和
    sum_of_sims = {} # 相似度总权重
    
    for person1 in prefs:
        if person1 == person: continue
        # 相似度小于0不考虑
        sim = similarity(prefs, person, person1)
        if sim < 0: continue
        
        for it in prefs[person1]:
            if it not in prefs[person] or prefs[person][it] == 0:
                totals.setdefault(it, 0)
                sum_of_sims.setdefault(it, 0)
                totals[it] +=  sim * prefs[person1][it]
                sum_of_sims[it] += sim
    # 统计平均相似度
    scores = [(total / sum_of_sims[it], it) for it, total in totals.items()]
    scores.sort(reverse=True)
    return scores

### 输出测试

In [32]:
print(get_recommendations(critics, 'Toby'))

[(3.3477895267131013, 'The Night Listener'), (2.8325499182641614, 'Lady in the Water'), (2.5309807037655645, 'Just My Luck')]


## 基于物品的匹配
- 以上的做法是基于用户的匹配，接下来要做的是基于物品的匹配：找出相似的物品

### 将prefs转置，基于物的索引
- 重复利用了前面的函数，对于变量名可能有些不相符

In [43]:
def transform_prefs(prefs):
    res = {}
    for person, item in prefs.items():
        for it in item:
            res.setdefault(it, {})
            res[it][person] = item[it]
    return res

moives = transform_prefs(critics)

### 输出测试

In [44]:
print(top_matched_people(moives, 'Superman Returns'))

[(0.6579516949597695, 'You, Me and Dupree'), (0.4879500364742689, 'Lady in the Water'), (0.11180339887498941, 'Snakes on a Plane'), (-0.1798471947990544, 'The Night Listener'), (-0.42289003161103106, 'Just My Luck')]


## 真实数据实战：电影推荐

### 加载数据

In [63]:
import pandas as pd

def load_data(path='./ml-latest-small'):
    movies = {}
    movies_data = pd.read_csv(path + '/movies.csv')
    
    for index, row in movies_data.iterrows():
        movies[int(row['movieId'])] = row['title']
    
    critics = {}
    critics_data = pd.read_csv(path + '/ratings.csv')
    
    for index, row in critics_data.iterrows():
        key1 = int(row['userId'])
        key2 = movies[int(row['movieId'])]
        critics.setdefault(key1, {})
        critics[key1][key2] = row['rating']
    
    return critics

critics = load_data()

### 输出测试

In [70]:
# 用户id为12的用户的评分清单
print(critics[12])


{'What About Bob? (1991)': 3.0, 'Army of Darkness (1993)': 5.0, 'Mad About Mambo (2000)': 2.0, 'Cat Ballou (1965)': 3.0, 'Saving Grace (2000)': 4.0, 'Sleeper (1973)': 3.0, 'Searching for Bobby Fischer (1993)': 1.0, 'Footloose (1984)': 1.0, 'Princess Bride, The (1987)': 1.0, "Bug's Life, A (1998)": 2.0, 'House Party (1990)': 2.0, 'Star Trek II: The Wrath of Khan (1982)': 1.0, 'Blame It on Rio (1984)': 2.0, 'What Lies Beneath (2000)': 4.0, 'Planet of the Apes (1968)': 1.0, 'Six Degrees of Separation (1993)': 3.0, 'X-Men (2000)': 5.0, 'Steel Magnolias (1989)': 1.0, 'Replacements, The (2000)': 3.0, 'Retroactive (1997)': 4.0, 'Twister (1996)': 4.0, 'Love & Sex (2000)': 3.0, 'Boiler Room (2000)': 4.0, 'Shane (1953)': 2.0, 'Man Who Fell to Earth, The (1976)': 4.0, 'Texas Chainsaw Massacre 2, The (1986)': 4.0, 'Cell, The (2000)': 5.0, 'Anatomy of a Murder (1959)': 3.0, 'Alice in Wonderland (1951)': 2.0, 'Unbearable Lightness of Being, The (1988)': 1.0, 'Drowning Mona (2000)': 1.0, 'Deuce Bigal

### 输出推荐结果

In [68]:
# 用户id为87的用户的推荐结果
print(get_recommendations(critics, 87)[:30])

[(5.0, 'Picture Bride (Bijo photo) (1994)'), (5.0, 'Patience Stone, The (2012)'), (5.0, 'Funhouse, The (1981)'), (5.0, 'Dylan Moran: Monster (2004)'), (4.9999999999999991, "Endurance: Shackleton's Legendary Antarctic Expedition, The (2000)"), (4.9109101827977186, 'Soul Kitchen (2009)'), (4.5275114886506209, 'Eye of the Needle (1981)'), (4.5133989659689631, 'Cypher (2002)'), (4.5, 'Valentin (Valentín) (2002)'), (4.5, "Twelve O'Clock High (1949)"), (4.5, 'Take This Waltz (2011)'), (4.5, 'Summer Storm (Sommersturm) (2004)'), (4.5, 'Snoopy Come Home (1972)'), (4.5, 'Elizabeth: The Golden Age (2007)'), (4.4436687402836705, 'Changeling (2008)'), (4.3861951815587101, 'Harvey Girls, The (1946)'), (4.3698999036829598, 'Walking and Talking (1996)'), (4.357478650478102, 'Taxi Driver (1976)'), (4.3221106092610828, 'Vertigo (1958)'), (4.3068443301050641, 'Red Balloon, The (Ballon rouge, Le) (1956)'), (4.2764527771062211, 'Nadja (1994)'), (4.1701116922033332, 'More (1998)'), (4.1645369795931249, 'My

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in double_scalars
